In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime
from datetime import timedelta
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
import json
from IPython.display import display
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)


%matplotlib inline

# Load data

In [2]:
path = '/Volumes/GoogleDrive/My Drive/wish_local/DSF-1626 Impact of SMS exp v1 by country'

In [3]:
df = pd.read_csv(f'{path}/Data/1196772236-by-country.csv')

df.head()

,country,bucket,dorders,orders,orsers_with_num_sms,avg_num_sms,avg_num_sms_non_pickup,cnt_pickup_14d,pickup_perc_14d,pickup_perc_14d_txn,pickup_gmv_perc_14d,avg_pickup_lead_time,std_pickup_lead_time,cnt_pickup_21d,pickup_perc_21d,pickup_gmv_perc_21d
0,AU,ignore,27951,27951,88,0.004257,0.002727,22817,0.816321,0.775626,0.812654,6.200100,11.072461,23816,0.852063,0.848954
1,AU,control,5383,5383,196,0.086197,0.591203,4610,0.856400,0.829349,0.860958,5.785303,10.171241,4759,0.884080,0.886427
2,AU,show,5685,5685,5682,2.111697,6.662362,5143,0.904661,0.900641,0.929258,4.071268,6.794335,5352,0.941425,0.954303
3,BR,ignore,71188,71188,384,0.007796,0.000231,58188,0.817385,0.795551,0.835041,6.139760,11.275228,60873,0.855102,0.866601
4,BR,control,23775,23775,68,0.004248,0.009712,20686,0.870074,0.850377,0.883155,5.220154,9.036987,21498,0.904227,0.910513


# pickup rate

## calculate diff

In [6]:
def show_control_diff(x, data, c, diff_pct=False):
    """
    x: row
    data: original dataframe
    c: column to calculate diff
    diff_pct: False is return absoluate diff, True is to return diff %
    """
    if 'show' not in x['bucket']:
        return
    if 'show' in x['bucket']:
        control_value = data[
                             (data['country']==x['country']) & \
                             (data['bucket']=='control')][c].iloc[0]
        show_value = x[c]
        if diff_pct:
            return (show_value - control_value) / control_value
        else:
            return show_value - control_value

In [20]:
df['pickup_perc_14d_diff'] = df.apply(lambda x: \
                show_control_diff(x, df, c='pickup_perc_14d', diff_pct=False), axis=1)
df[['country', 'bucket', 'dorders', 'pickup_perc_14d', 'pickup_perc_14d_diff']]

,country,bucket,dorders,pickup_perc_14d,pickup_perc_14d_diff
0,AU,ignore,27951,0.816321,NaN
1,AU,control,5383,0.856400,NaN
2,AU,show,5685,0.904661,0.048262
3,BR,ignore,71188,0.817385,NaN
4,BR,control,23775,0.870074,NaN
5,BR,show,24689,0.901940,0.031867
6,CA,ignore,123498,0.867844,NaN
7,CA,control,7814,0.891349,NaN
8,CA,show,7725,0.946926,0.055577
9,CH,ignore,16341,0.820207,NaN


## Proportion test

In [22]:
from statsmodels.stats.proportion import proportions_ztest
# https://www.statsmodels.org/devel/generated/statsmodels.stats.proportion.proportions_ztest.html

def show_control_prop_test(x, data, pct_col, nobs_col, is_count=False, alternative='two-sided'):
    """
    x: row
    data: original dataframe
    pct_col: column name of numerator or the percentage calcualted
    nobs_col: column name of denominator
    is_count: False if pct_col is percentage, True if pct_col is the integer (numerator)
    """
    if 'show' not in x['bucket']:
        return
    if 'show' in x['bucket']:
        n_control = data[
                         (data['country']==x['country']) & \
                         (data['bucket']=='control')][nobs_col].iloc[0]
        n_show = x[nobs_col]
        if is_count: 
            count_control = data[
                                 (data['country']==x['country']) & \
                                 (data['bucket']=='control')][pct_col].iloc[0]
            count_show = x[pct_col]
        else:
            count_control = int(round(data[
                                           (data['country']==x['country']) & \
                                           (data['bucket']=='control')][pct_col].iloc[0] * n_control,0))
            count_show = int(round(x[pct_col] * n_show, 0))
        
        stat, pval = proportions_ztest((count_control,count_show), (n_control,n_show), alternative=alternative)
        
        return round(pval, 6)

In [23]:
proportions_ztest((4610,5143), (5383,5685), alternative='two-sided')

(-7.842993225016693, 4.399317352504872e-15)

In [21]:
df['pickup_perc_14d'+'_pval'] = df.apply(lambda x: \
         show_control_prop_test(x, df, 'cnt_pickup_14d', 'dorders', is_count=True, alternative='two-sided')
         , axis=1)

df[['country', 'bucket', 'dorders', 'cnt_pickup_14d', 'pickup_perc_14d', 'pickup_perc_14d_diff', 'pickup_perc_14d_pval']]

,country,bucket,dorders,cnt_pickup_14d,pickup_perc_14d,pickup_perc_14d_diff,pickup_perc_14d_pval
0,AU,ignore,27951,22817,0.816321,NaN,NaN
1,AU,control,5383,4610,0.856400,NaN,NaN
2,AU,show,5685,5143,0.904661,0.048262,0.000000
3,BR,ignore,71188,58188,0.817385,NaN,NaN
4,BR,control,23775,20686,0.870074,NaN,NaN
5,BR,show,24689,22268,0.901940,0.031867,0.000000
6,CA,ignore,123498,107177,0.867844,NaN,NaN
7,CA,control,7814,6965,0.891349,NaN,NaN
8,CA,show,7725,7315,0.946926,0.055577,0.000000
9,CH,ignore,16341,13403,0.820207,NaN,NaN


# Pickup lead time

In [24]:
df.columns

Index(['country', 'bucket', 'dorders', 'orders', 'orsers_with_num_sms',
       'avg_num_sms', 'avg_num_sms_non_pickup', 'cnt_pickup_14d',
       'pickup_perc_14d', 'pickup_perc_14d_txn', 'pickup_gmv_perc_14d',
       'avg_pickup_lead_time', 'std_pickup_lead_time', 'cnt_pickup_21d',
       'pickup_perc_21d', 'pickup_gmv_perc_21d', 'pickup_perc_14d_diff',
       'pickup_perc_14d_pval'],
      dtype='object')

## calculate diff 

In [25]:
df['avg_pickup_lead_time_diff'] = df.apply(lambda x: \
                show_control_diff(x, df, c='avg_pickup_lead_time', diff_pct=False), axis=1)
df[['country', 'bucket', 'dorders', 'avg_pickup_lead_time', 'avg_pickup_lead_time_diff']]

,country,bucket,dorders,avg_pickup_lead_time,avg_pickup_lead_time_diff
0,AU,ignore,27951,6.200100,NaN
1,AU,control,5383,5.785303,NaN
2,AU,show,5685,4.071268,-1.714035
3,BR,ignore,71188,6.139760,NaN
4,BR,control,23775,5.220154,NaN
5,BR,show,24689,4.527757,-0.692397
6,CA,ignore,123498,4.627832,NaN
7,CA,control,7814,4.413365,NaN
8,CA,show,7725,3.665348,-0.748017
9,CH,ignore,16341,7.126374,NaN


## T test

In [26]:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html
# https://github.com/scipy/scipy/blob/v1.7.1/scipy/stats/stats.py#L5879-

from scipy.stats import distributions

def _unequal_var_ttest_denom(v1, n1, v2, n2):
    vn1 = v1 / n1
    vn2 = v2 / n2
    with np.errstate(divide='ignore', invalid='ignore'):
        dof = (vn1 + vn2)**2 / (vn1**2 / (n1 - 1) + vn2**2 / (n2 - 1))

    # If dof is undefined, variances are zero (assumes n1 > 0 & n2 > 0).
    # Hence it doesn't matter what dof is as long as it's not NaN.
    dof = np.where(np.isnan(dof), 1, dof)
    denom = np.sqrt(vn1 + vn2)
    return dof, denom

def _ttest_finish(dof, t, alternative):
    """Common code between all 3 t-test functions."""
    if alternative == 'less':
        prob = distributions.t.cdf(t, dof)
    elif alternative == 'greater':
        prob = distributions.t.sf(t, dof)
    elif alternative == 'two-sided':
        prob = 2 * distributions.t.sf(np.abs(t), dof)
    else:
        raise ValueError("alternative must be "
                         "'less', 'greater' or 'two-sided'")
    return t, prob

def _ttest_ind_from_stats(mean1, mean2, denom, dof, alternative):

    d = mean1 - mean2
    with np.errstate(divide='ignore', invalid='ignore'):
        t = np.divide(d, denom)
    t, prob = _ttest_finish(dof, t, alternative)

    return (t, prob)



def unequal_var_ttest(v1, v2, n1, n2, m1, m2, alternative='two-sided'):
    dof, denom = _unequal_var_ttest_denom(v1, n1, v2, n2)
    t, pval = _ttest_ind_from_stats(m1, m2, denom, dof, alternative)
    
    return t, pval

In [64]:
def show_control_t_test(x, data, avg_col, std_col, n_col, alternative='two-sided'):
    """
    x: row
    data: original dataframe
    """
    if 'show' not in x['bucket']:
        return
    if 'show' in x['bucket']:
        v_control = data[
                         (data['country']==x['country']) & \
                        (data['bucket']=='control')][std_col].iloc[0] ** 2
        v_show = x[std_col] ** 2
        
        n_control = data[
                         (data['country']==x['country']) & \
                        (data['bucket']=='control')][n_col].iloc[0]
        n_show = x[n_col]
        
        m_control = data[
                         (data['country']==x['country']) & \
                        (data['bucket']=='control')][avg_col].iloc[0]
        m_show = x[avg_col]
        
        t, pval = unequal_var_ttest(v_control, v_show, n_control, n_show, m_control, m_show, alternative)
        
        return round(t,1), pval#(str(round(pval, 5)), round(t, 2))

In [65]:
c = 'pickup_lead_time'
df[c+'_pval'] = df.apply(lambda x: \
                                 show_control_t_test(x, data=df, \
                                                     avg_col='avg_'+c, \
                                                     std_col='std_'+c, \
                                                     n_col='dorders', \
                                                     alternative='two-sided') , axis=1)

In [66]:
df[['country', 'bucket', 'dorders', 'avg_pickup_lead_time', 'avg_pickup_lead_time_diff', c+'_pval']]

,country,bucket,dorders,avg_pickup_lead_time,avg_pickup_lead_time_diff,pickup_lead_time_pval
0,AU,ignore,27951,6.200100,NaN,None
1,AU,control,5383,5.785303,NaN,None
2,AU,show,5685,4.071268,-1.714035,"(10.4, 4.820338476659649e-25)"
3,BR,ignore,71188,6.139760,NaN,None
4,BR,control,23775,5.220154,NaN,None
5,BR,show,24689,4.527757,-0.692397,"(8.7, 2.2018894818426517e-18)"
6,CA,ignore,123498,4.627832,NaN,None
7,CA,control,7814,4.413365,NaN,None
8,CA,show,7725,3.665348,-0.748017,"(6.2, 5.257299861526117e-10)"
9,CH,ignore,16341,7.126374,NaN,None
